In [1]:
import urllib.request
import zipfile

In [10]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'

validation_url = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'

# Run this only if you <font color='red'>don't have images</font> in your current directory

In [6]:
urllib.request.urlretrieve(url, file_name)
with zipfile.ZipFile(file_name) as f:
    f.extractall(training_dir)

In [12]:
urllib.request.urlretrieve(validation_url, validation_file_name)
with zipfile.ZipFile(validation_file_name) as f:
    f.extractall(validation_dir)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop

2024-01-30 13:24:29.255576: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 13:24:29.257708: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 13:24:29.293588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 13:24:29.293622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 13:24:29.294671: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [16]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    class_mode="binary"
)

Found 256 images belonging to 2 classes.


In [5]:
model = Sequential([
    Conv2D(16, (3, 3), activation="relu", input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

In [7]:
model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)

In [19]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15
)

Epoch 1/15
33/33 [==============================] - 16s 473ms/step - loss: 6.8595e-07 - accuracy: 1.0000 - val_loss: 4.4735 - val_accuracy: 0.7852
Epoch 2/15
33/33 [==============================] - 17s 507ms/step - loss: 6.1557e-07 - accuracy: 1.0000 - val_loss: 4.4856 - val_accuracy: 0.7852
Epoch 3/15
33/33 [==============================] - 16s 490ms/step - loss: 5.8680e-07 - accuracy: 1.0000 - val_loss: 4.5166 - val_accuracy: 0.7852
Epoch 4/15
33/33 [==============================] - 15s 459ms/step - loss: 5.3446e-07 - accuracy: 1.0000 - val_loss: 4.5110 - val_accuracy: 0.7852
Epoch 5/15
33/33 [==============================] - 16s 475ms/step - loss: 5.0302e-07 - accuracy: 1.0000 - val_loss: 4.5259 - val_accuracy: 0.7852
Epoch 6/15
33/33 [==============================] - 15s 465ms/step - loss: 4.6721e-07 - accuracy: 1.0000 - val_loss: 4.5683 - val_accuracy: 0.7852
Epoch 7/15
33/33 [==============================] - 16s 492ms/step - loss: 4.4176e-07 - accuracy: 1.0000 - val_loss: 4